■SUUMOのホームページから賃貸情報を取得する

In [1]:
#おまじない
#importする順番はアルファベット順が良い！
from bs4 import BeautifulSoup
import requests

In [2]:
#変数URLにSUUMOのホームページURLを格納する
url = 'https://suumo.jp/chintai/osaka/sc_osakashinishi/?page={}'

#アクセスするためのURLをtarget_urlに格納する
target_url = url.format(1)

#print()してtarget_urlを確認する
print(target_url)

https://suumo.jp/chintai/osaka/sc_osakashinishi/?page=1


In [3]:
#target_urlへのアクセス結果を変数rに格納する
r = requests.get(target_url)

#取得結果を解析してsoupに格納する
soup = BeautifulSoup(r.text)

■物件情報の取得

In [4]:
#HTMLを確認→それぞれの賃貸情報はcassetteitemに格納されている
contents = soup.find_all('div',class_='cassetteitem')

In [5]:
#find_allで取得した結果はリスト形式になっているのでlenで要素数を確認。1pあたり２０件
len(contents)

20

In [6]:
#まずは一つ取り出す！
content = contents[0]

In [7]:
#物件・建物情報を変数detailに格納
detail = content.find('div',class_='cassetteitem-detail')

In [8]:
#変数titleに物件名を格納,text情報だけ抜くので.text
title = detail.find('div',class_='cassetteitem_content-title').text

#変数addressに住所を格納
address = detail.find('li',class_='cassetteitem_detail-col1').text

#変数accessにアクセス情報を格納
access = detail.find('li',class_='cassetteitem_detail-col2').text

#変数ageに築年数を格納
age = detail.find('li',class_='cassetteitem_detail-col3').text

In [9]:
title,address,access,age

('ルシエルみなと通り\u3000旭化成ヘーベルメゾンHEBEL HAUS',
 '大阪府大阪市西区九条南２',
 '\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大阪環状線/大正駅 歩12分\n',
 '\n築8年\n4階建\n')

■部屋情報の取得

In [10]:
#各部屋の情報を変数tableに格納
table = content.find('div',class_='cassetteitem-item')

In [11]:
#変数tableから全てのtrタグを取得、変数tr_tagsに格納
tr_tags = table.find_all('tr',class_='js-cassette_link')

#tr_tagsの中から最初の一つだけtr_tagに格納
tr_tag = tr_tags[0]

In [12]:
#trの中身確認。tdの３〜６番目に情報が格納されている。
#find_allで取得した結果はリスト形式で出てくるので、スライスを使って欲しい部分を取り出す

#変数floor,price,first_fee,capacity

floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]

In [13]:
#unpack→パックされたいた物を解除する方法！
l = [1,2,3,4,5]
a,b = l[2:4]
print(a)
print(b)

3
4


In [14]:
#さらにliが入っているので分解
#変数feeとmanagement_feeに賃料と管理費を格納
fee,management_fee = price.find_all('li')

#変数depositとgratuityに敷金、礼金を格納
deposit,gratuity = first_fee.find_all('li')

#変数madoriとmensekiに間取りと面積を格納
madori,menseki = capacity.find_all('li')

In [15]:
print(fee)
print(management_fee)
print()
print(deposit)
print(gratuity)
print()
print(madori)
print(menseki)

<li><span class="cassetteitem_price cassetteitem_price--rent"><span class="cassetteitem_other-emphasis ui-text--bold">8.3万円</span></span></li>
<li><span class="cassetteitem_price cassetteitem_price--administration">6000円</span></li>

<li><span class="cassetteitem_price cassetteitem_price--deposit">5万円</span></li>
<li><span class="cassetteitem_price cassetteitem_price--gratuity">16.6万円</span></li>

<li><span class="cassetteitem_madori">1LDK</span></li>
<li><span class="cassetteitem_menseki">46.83m<sup>2</sup></span></li>


■取得した情報を辞書に格納する

In [16]:
#あとでデータを扱いやすくするために辞書型に格納する！

d = {
    'title':title,
    'address':address,
    'access':access,
    'age':age,
    'floor':floor.text,
    'fee':fee.text,
    'management_fee':management_fee.text,
    'deposit':deposit.text,
    'gratuity':gratuity.text,
    'madori':madori.text,
    'menseki':menseki.text
}

In [17]:
d

{'title': 'ルシエルみなと通り\u3000旭化成ヘーベルメゾンHEBEL HAUS',
 'address': '大阪府大阪市西区九条南２',
 'access': '\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大阪環状線/大正駅 歩12分\n',
 'age': '\n築8年\n4階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t3階',
 'fee': '8.3万円',
 'management_fee': '6000円',
 'deposit': '5万円',
 'gratuity': '16.6万円',
 'madori': '1LDK',
 'menseki': '46.83m2'}

■1ページ目から全ての情報を取得する

In [18]:
#変数d_listに空のリストを作成する
d_list = []

#全ての物件情報（２０件）を取得する
contents = soup.find_all('div',class_='cassetteitem')

#各物件情報をforループで取得する
for content in contents:
    
    #物件情報と部屋情報を取得
    detail = content.find('div',class_='cassetteitem-detail')
    table = content.find('div',class_='cassetteitem-item')
    
    #物件情報から必要な情報を取得
    title = detail.find('div',class_='cassetteitem_content-title').text
    address = detail.find('li',class_='cassetteitem_detail-col1').text
    access = detail.find('li',class_='cassetteitem_detail-col2').text
    age = detail.find('li',class_='cassetteitem_detail-col3').text
    
    #部屋情報のブロックから各部屋情報を取得
    tr_tags = table.find_all('tr',class_='js-cassette_link')
    
    #各部屋情報をforループで取得する
    for tr_tag in tr_tags:
        
        #部屋情報の行から欲しい情報を取得
        floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]
        
        #さらに細かい情報を取得
        fee,management_fee = price.find_all('li')
        deposit,gratuity = first_fee.find_all('li')
        madori,menseki = capacity.find_all('li')
        
        #取得した全ての情報を辞書に格納
        d = {
            'title':title,
            'address':address,
            'access':access,
            'age':age,
            'floor':floor.text,
            'fee':fee.text,
            'management_fee':management_fee.text,
            'deposit':deposit.text,
            'gratuity':gratuity.text,
            'madori':madori.text,
            'menseki':menseki.text
        }
        
        #取得した辞書をd_listに格納する！
        
        d_list.append(d)

In [19]:
#printを使うと見辛いのでpprintをインポートする
from pprint import pprint

In [20]:
pprint(d_list[0])
print()
pprint(d_list[1])

{'access': '\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大阪環状線/大正駅 歩12分\n',
 'address': '大阪府大阪市西区九条南２',
 'age': '\n築8年\n4階建\n',
 'deposit': '5万円',
 'fee': '8.3万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t3階',
 'gratuity': '16.6万円',
 'madori': '1LDK',
 'management_fee': '6000円',
 'menseki': '46.83m2',
 'title': 'ルシエルみなと通り\u3000旭化成ヘーベルメゾンHEBEL HAUS'}

{'access': '\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ橋線/四ツ橋駅 歩15分\n',
 'address': '大阪府大阪市西区立売堀４',
 'age': '\n築3年\n15階建\n',
 'deposit': '-',
 'fee': '6.15万円',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t4階',
 'gratuity': '6.15万円',
 'madori': '1K',
 'management_fee': '7250円',
 'menseki': '23.02m2',
 'title': 'プレサンス立売堀ベルヴィル'}


■複数ページから情報取得する

In [21]:
#ライブラリのインポート
from time import sleep

In [22]:
#1　アクセスするURLを設定する　→ここを２、３・・・と変更していけばよい！
#2 Requestsを使って設定したURLにアクセスする
#3 取得したHTMLをBeautifulSoupをで解析する
#4 全ての物件情報を取得する（２０件）
#5 各物件情報から物件の詳細と各部屋情報を取得する
#6　それぞれを解析する
#7　解析した結果を辞書に格納する

In [23]:
#変数URLにSUUMOのホームページURLを格納する
url = 'https://suumo.jp/chintai/osaka/sc_osakashinishi/?page={}'
d_list = []

In [24]:
#1〜３をループする

for i in range(1,4):
    print('d_listの大きさ:', len(d_list))
    
    #変数target_urlにアクセス先のURLを格納する
    target_url = url.format(i)
    
    print(target_url) 
    
    #target_urlへのアクセス結果を変数rに格納する→ここでデータとってきてる！
    r = requests.get(target_url)
    
    sleep(1)
    
    #取得結果を解析してsoupに格納する
    soup = BeautifulSoup(r.text)
    
    #変数d_listに空のリストを作成する
    #d_list = []→これ不要！！！

    #全ての物件情報（２０件）を取得する
    contents = soup.find_all('div',class_='cassetteitem')

    #各物件情報をforループで取得する
    for content in contents:
    
        #物件情報と部屋情報を取得
        detail = content.find('div',class_='cassetteitem-detail')
        table = content.find('div',class_='cassetteitem-item')

        #物件情報から必要な情報を取得
        title = detail.find('div',class_='cassetteitem_content-title').text
        address = detail.find('li',class_='cassetteitem_detail-col1').text
        access = detail.find('li',class_='cassetteitem_detail-col2').text
        age = detail.find('li',class_='cassetteitem_detail-col3').text

        #部屋情報のブロックから各部屋情報を取得
        tr_tags = table.find_all('tr',class_='js-cassette_link')

        #各部屋情報をforループで取得する
        for tr_tag in tr_tags:

            #部屋情報の行から欲しい情報を取得
            floor,price,first_fee,capacity = tr_tag.find_all('td')[2:6]

            #さらに細かい情報を取得
            fee,management_fee = price.find_all('li')
            deposit,gratuity = first_fee.find_all('li')
            madori,menseki = capacity.find_all('li')

            #取得した全ての情報を辞書に格納
            d = {
                'title':title,
                'address':address,
                'access':access,
                'age':age,
                'floor':floor.text,
                'fee':fee.text,
                'management_fee':management_fee.text,
                'deposit':deposit.text,
                'gratuity':gratuity.text,
                'madori':madori.text,
                'menseki':menseki.text
            }

            #取得した辞書をd_listに格納する！

            d_list.append(d)

d_listの大きさ: 0
https://suumo.jp/chintai/osaka/sc_osakashinishi/?page=1
d_listの大きさ: 216
https://suumo.jp/chintai/osaka/sc_osakashinishi/?page=2
d_listの大きさ: 457
https://suumo.jp/chintai/osaka/sc_osakashinishi/?page=3


In [25]:
#最後の物件を確認！
d_list[-1]

{'title': 'レフィシアコート江戸堀',
 'address': '大阪府大阪市西区江戸堀１',
 'access': '\n地下鉄四つ橋線/肥後橋駅 歩8分\n地下鉄御堂筋線/本町駅 歩11分\n地下鉄千日前線/阿波座駅 歩17分\n',
 'age': '\n築3年\n15階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t6階',
 'fee': '12万円',
 'management_fee': '8000円',
 'deposit': '2万円',
 'gratuity': '25万円',
 'madori': '1LDK',
 'menseki': '46.87m2'}

■スクレイピング結果をCSVに保存する

In [26]:
#ライブラリのインポート
import pandas as pd

In [27]:
d_list[:3]

[{'title': 'ルシエルみなと通り\u3000旭化成ヘーベルメゾンHEBEL HAUS',
  'address': '大阪府大阪市西区九条南２',
  'access': '\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大阪環状線/大正駅 歩12分\n',
  'age': '\n築8年\n4階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t3階',
  'fee': '8.3万円',
  'management_fee': '6000円',
  'deposit': '5万円',
  'gratuity': '16.6万円',
  'madori': '1LDK',
  'menseki': '46.83m2'},
 {'title': 'プレサンス立売堀ベルヴィル',
  'address': '大阪府大阪市西区立売堀４',
  'access': '\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ橋線/四ツ橋駅 歩15分\n',
  'age': '\n築3年\n15階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t4階',
  'fee': '6.15万円',
  'management_fee': '7250円',
  'deposit': '-',
  'gratuity': '6.15万円',
  'madori': '1K',
  'menseki': '23.02m2'},
 {'title': 'プレサンス立売堀ベルヴィル',
  'address': '大阪府大阪市西区立売堀４',
  'access': '\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ橋線/四ツ橋駅 歩15分\n',
  'age': '\n築3年\n15階建\n',
  'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t2階',
  'fee': '6.38万円',
  'management_fee': '7310円',
  'deposit': '-',
  'gratuity': '6.38万円',
  'madori': '

■取得したデータを保存する

In [28]:
#取得したデータ結果を表形式に変える！
#headで先頭業を確認

df = pd.DataFrame(d_list)
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪府大阪市西区九条南２,\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大...,\n築8年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,8.3万円,6000円,5万円,16.6万円,1LDK,46.83m2
1,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,6.15万円,7250円,-,6.15万円,1K,23.02m2
2,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,6.38万円,7310円,-,6.38万円,1K,23.2m2
3,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t10階,6.53万円,7250円,-,6.53万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t11階,6.56万円,7250円,-,-,1K,23.2m2


In [29]:
#uniqueで重複なく取り出せる→２０件×3ページになっているか確認
len(df.title.unique())

60

In [30]:
#encordinghはエクセル文字化け対策！
df.to_csv('test.csv',index=None,encoding='utf-8-sig')

■スプレッドシートのインポート(0614)

In [9]:
!pip3 install gspread
!pip3 install oauth2client

In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [11]:
#jsonfファイルを使って認証情報を取得する！
SCOPES = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'tenacious-works-389907-13b55bdb7d6b.json'

In [12]:
#秘密鍵jsonファイルへのパス、APIの情報
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE,SCOPES)

In [59]:
gs = gspread.authorize(credentials)

#worksheetの名前はファイル名じゃなくてシート名なので注意！
SP_SHEET_KEY = '11wt6LvbrqjHywgNaGMsLaZKAj7n5ovaO1G4qPzj8jDE'
worksheet = gs.open_by_key(SP_SHEET_KEY).worksheet("suumo_nishiku")

In [60]:
worksheet

<Worksheet 'suumo_nishiku' id:849754097>

In [58]:
print(worksheet.acell("G2").value)

6000円


In [ ]:
■読み込んだデータの集計

In [24]:
import pandas as pd

In [61]:
#ここがよくわからん・・・・・・・
workbook = gs.open_by_key(SP_SHEET_KEY)
worksheet = workbook.worksheet("suumo_nishiku")

In [76]:
print(workbook.title)
print(workbook.id)

suumo_nishiku
11wt6LvbrqjHywgNaGMsLaZKAj7n5ovaO1G4qPzj8jDE


In [75]:
print(worksheet)

<Worksheet 'suumo_nishiku' id:849754097>


In [77]:
df = pd.DataFrame(worksheet.get_all_values())
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
1,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪府大阪市西区九条南２,\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大...,\n築8年\n4階建\n,\n\t\t\t\t\t\t\t\t\t\t\t3階,8.3万円,6000円,5万円,16.6万円,1LDK,46.83m2
2,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t4階,6.15万円,7250円,-,6.15万円,1K,23.02m2
3,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t2階,6.38万円,7310円,-,6.38万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t10階,6.53万円,7250円,-,6.53万円,1K,23.2m2


In [78]:
#データフレームの最初の行のデータをカラムに設定する
df.columns = df.iloc[0]
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
1,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪府大阪市西区九条南２,\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大...,\n築8年\n4階建\n,\n\t\t\t\t\t\t\t\t\t\t\t3階,8.3万円,6000円,5万円,16.6万円,1LDK,46.83m2
2,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t4階,6.15万円,7250円,-,6.15万円,1K,23.02m2
3,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t2階,6.38万円,7310円,-,6.38万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t10階,6.53万円,7250円,-,6.53万円,1K,23.2m2


In [79]:
#１行目とカラムのデータが同じなので、１行目をdrop
df = df.drop(df.index[[0]])
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
1,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪府大阪市西区九条南２,\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大...,\n築8年\n4階建\n,\n\t\t\t\t\t\t\t\t\t\t\t3階,8.3万円,6000円,5万円,16.6万円,1LDK,46.83m2
2,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t4階,6.15万円,7250円,-,6.15万円,1K,23.02m2
3,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t2階,6.38万円,7310円,-,6.38万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t10階,6.53万円,7250円,-,6.53万円,1K,23.2m2
5,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t11階,6.56万円,7250円,-,-,1K,23.2m2


In [80]:
#データ型の確認
df.dtypes

0
title             object
address           object
access            object
age               object
floor             object
fee               object
management_fee    object
deposit           object
gratuity          object
madori            object
menseki           object
dtype: object

In [81]:
#feeの万円を削除しobject型からfloat型へ変換

df['fee'] = df['fee'].str.replace("万円","")
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
1,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪府大阪市西区九条南２,\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大...,\n築8年\n4階建\n,\n\t\t\t\t\t\t\t\t\t\t\t3階,8.3,6000円,5万円,16.6万円,1LDK,46.83m2
2,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t4階,6.15,7250円,-,6.15万円,1K,23.02m2
3,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t2階,6.38,7310円,-,6.38万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t10階,6.53,7250円,-,6.53万円,1K,23.2m2
5,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,\n\t\t\t\t\t\t\t\t\t\t\t11階,6.56,7250円,-,-,1K,23.2m2


In [82]:
df['fee'] = df['fee'].astype(float)
df.dtypes

0
title              object
address            object
access             object
age                object
floor              object
fee               float64
management_fee     object
deposit            object
gratuity           object
madori             object
menseki            object
dtype: object

In [83]:
#家賃を数値に変換して平均値出してみる！
df['fee'].mean()

7.664453551912573

In [84]:
#階数の/tを削除

df['floor'] = df['floor'].str.replace("\n","")
df['floor'] = df['floor'].str.replace("\r","")
df['floor'] = df['floor'].str.replace("\t","")
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
1,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪府大阪市西区九条南２,\n地下鉄中央線/九条駅 歩5分\n地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分\nＪＲ大...,\n築8年\n4階建\n,3階,8.30,6000円,5万円,16.6万円,1LDK,46.83m2
2,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,4階,6.15,7250円,-,6.15万円,1K,23.02m2
3,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,2階,6.38,7310円,-,6.38万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,10階,6.53,7250円,-,6.53万円,1K,23.2m2
5,プレサンス立売堀ベルヴィル,大阪府大阪市西区立売堀４,\n地下鉄千日前線/阿波座駅 歩5分\n地下鉄長堀鶴見緑地線/西長堀駅 歩6分\n地下鉄四つ...,\n築3年\n15階建\n,11階,6.56,7250円,-,-,1K,23.2m2


In [85]:
#access,ageの/nを削除、アドレスの大阪府削除

df['access'] = df['access'].str.replace("\n"," ")
df['age'] = df['age'].str.replace("\n"," ")
df['address'] = df['address'].str.replace("大阪府","")
df.head()

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
1,ルシエルみなと通り 旭化成ヘーベルメゾンHEBEL HAUS,大阪市西区九条南２,地下鉄中央線/九条駅 歩5分 地下鉄長堀鶴見緑地線/ドーム前千代崎駅 歩7分 ＪＲ大阪環状...,築8年 4階建,3階,8.30,6000円,5万円,16.6万円,1LDK,46.83m2
2,プレサンス立売堀ベルヴィル,大阪市西区立売堀４,地下鉄千日前線/阿波座駅 歩5分 地下鉄長堀鶴見緑地線/西長堀駅 歩6分 地下鉄四つ橋線/...,築3年 15階建,4階,6.15,7250円,-,6.15万円,1K,23.02m2
3,プレサンス立売堀ベルヴィル,大阪市西区立売堀４,地下鉄千日前線/阿波座駅 歩5分 地下鉄長堀鶴見緑地線/西長堀駅 歩6分 地下鉄四つ橋線/...,築3年 15階建,2階,6.38,7310円,-,6.38万円,1K,23.2m2
4,プレサンス立売堀ベルヴィル,大阪市西区立売堀４,地下鉄千日前線/阿波座駅 歩5分 地下鉄長堀鶴見緑地線/西長堀駅 歩6分 地下鉄四つ橋線/...,築3年 15階建,10階,6.53,7250円,-,6.53万円,1K,23.2m2
5,プレサンス立売堀ベルヴィル,大阪市西区立売堀４,地下鉄千日前線/阿波座駅 歩5分 地下鉄長堀鶴見緑地線/西長堀駅 歩6分 地下鉄四つ橋線/...,築3年 15階建,11階,6.56,7250円,-,-,1K,23.2m2


■スプレッドシートに書き出す

In [87]:
!pip3 install gspread_dataframe

In [93]:
from gspread_dataframe import set_with_dataframe

In [94]:
workbook.add_worksheet(title = "new_suumo_nishiku", rows=50, cols=11)

APIError: {'code': 400, 'message': 'Invalid requests[0].addSheet: A sheet with the name "new_suumo_nishiku" already exists. Please enter another name.', 'status': 'INVALID_ARGUMENT'}

In [96]:
set_with_dataframe(workbook.worksheet("new_suumo_nishiku"),df, include_index = True)